In [7]:
from PIL import Image
import os
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16

DATADIR = 'C:/Users/mine/Desktop/natural'
CATEGORIES = ['buildings', 'forest', 'glaciers', 'mountains', 'sea']
img_size = 128

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add new trainable layers on top of the pre-trained model
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(len(CATEGORIES), activation='softmax')(x)

# Define the new model
model = tf.keras.models.Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Load and preprocess the data
def load_data():
    training_data = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                new_array = cv2.resize(img_array, (img_size, img_size))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass
    x = np.array([i[0] for i in training_data]) / 255.0
    y = np.array([i[1] for i in training_data])
    y = tf.keras.utils.to_categorical(y, len(CATEGORIES))
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
    return x_train, x_val, y_train, y_val

x_train, x_val, y_train, y_val = load_data()

# Train the model
model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))


Epoch 1/10
49/49 [==============================] - 835s 17s/step - loss: 1.1988 - accuracy: 0.6865 - val_loss: 0.5301 - val_accuracy: 0.7928
Epoch 2/10
49/49 [==============================] - 832s 17s/step - loss: 0.2970 - accuracy: 0.9032 - val_loss: 0.4331 - val_accuracy: 0.8542
Epoch 3/10
49/49 [==============================] - 827s 17s/step - loss: 0.1951 - accuracy: 0.9314 - val_loss: 0.4736 - val_accuracy: 0.8210
Epoch 4/10
49/49 [==============================] - 833s 17s/step - loss: 0.0926 - accuracy: 0.9769 - val_loss: 0.4124 - val_accuracy: 0.8747
Epoch 5/10
49/49 [==============================] - 826s 17s/step - loss: 0.0452 - accuracy: 0.9929 - val_loss: 0.3954 - val_accuracy: 0.8440
Epoch 6/10
49/49 [==============================] - 825s 17s/step - loss: 0.0216 - accuracy: 0.9994 - val_loss: 0.4433 - val_accuracy: 0.8747
Epoch 7/10
49/49 [==============================] - 827s 17s/step - loss: 0.0143 - accuracy: 1.0000 - val_loss: 0.4166 - val_accuracy: 0.8696
Epoch 

In [13]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_val, y_val, verbose=0)

# Print the test loss and accuracy
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

Test loss: 0.42809000611305237
Test accuracy: 0.8644501566886902


In [14]:
model.save('my_model.h5')

In [22]:
import numpy as np
from PIL import Image
from keras.models import load_model

# Load your trained CNN model
model = load_model('my_model.h5')

# Load the image you want to classify
image = Image.open(r'C:\Users\mine\Desktop\natural\mountains\images344.jpg')

# Preprocess the image
image = image.resize((128, 128))
image = np.array(image) / 255.0
image = image.reshape((1, 128, 128, 3))

# Make a prediction
preds = model.predict(image)

# Find the index of the highest probability value
pred_class = np.argmax(preds)

# Map the predicted index to the corresponding class label
classes = ['buildings', 'forest', 'sea', 'glaciers', 'mountains']
pred_label = classes[pred_class]

# Display the result
print('The predicted class of the image is:', pred_label)

1/1 [==============================] - 1s 977ms/step
The predicted class of the image is: glaciers
